In [2]:
import pandas as pd
pd.options.mode.chained_assignment = None


### In data processing we first try and bring data to a accpetable format , pandas is used to load and modify data so we can use it for further analysis



In [3]:
sentiment_info = pd.read_csv('data/train.tsv',delimiter='\t',encoding='utf-8')

In [4]:
sentiment_info.head()


,PhraseId,SentenceId,Phrase,Sentiment
0,1,1,A series of escapades demonstrating the adage ...,1
1,2,1,A series of escapades demonstrating the adage ...,2
2,3,1,A series,2
3,4,1,A,2
4,5,1,series,2


## As the data has 0 to 4 degrees of sentiment , we try to keep only the extreme classes , so are only concerned about 0 and 4 

In [5]:
senti_short = sentiment_info[(sentiment_info['Sentiment']==0)|(sentiment_info['Sentiment']==4)]


In [6]:

senti_short.head()


,PhraseId,SentenceId,Phrase,Sentiment
63,64,2,"This quiet , introspective and entertaining in...",4
66,67,2,"quiet , introspective and entertaining indepen...",4
74,75,2,entertaining,4
77,78,2,is worth seeking,4
101,102,3,would have a hard time sitting through this one,0


## We create a new column in our data and convert 4 as 1 and 0 remains the same

In [7]:
senti_short['Bincol']= senti_short.apply(lambda x:0 if x['Sentiment']==0 else 1,axis=1)

In [8]:
senti_short.head()


,PhraseId,SentenceId,Phrase,Sentiment,Bincol
63,64,2,"This quiet , introspective and entertaining in...",4,1
66,67,2,"quiet , introspective and entertaining indepen...",4,1
74,75,2,entertaining,4,1
77,78,2,is worth seeking,4,1
101,102,3,would have a hard time sitting through this one,0,0


In [9]:
import numpy as np
import keras 
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer
from sklearn.feature_extraction.text import CountVectorizer
from keras.utils.np_utils import to_categorical
from sklearn.model_selection import train_test_split

import re

Using TensorFlow backend.


In [10]:
max_fatures = 2500
tokenizer = Tokenizer(num_words=max_fatures, split=' ')
tokenizer.fit_on_texts(senti_short['Phrase'].values)
X = tokenizer.texts_to_sequences(senti_short['Phrase'].values)
X = pad_sequences(X)

In [11]:
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D

In [12]:
embed_dim = 256
lstm_out = 128

model = Sequential()
model.add(Embedding(max_fatures, embed_dim,input_length = X.shape[1]))
model.add(SpatialDropout1D(0.4))
model.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(2,activation='softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 45, 256)           640000    
_________________________________________________________________
spatial_dropout1d_1 (Spatial (None, 45, 256)           0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 128)               197120    
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 258       
Total params: 837,378
Trainable params: 837,378
Non-trainable params: 0
_________________________________________________________________
None


In [13]:
Y = pd.get_dummies(senti_short['Sentiment']).values
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.33, random_state = 42)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

(10906, 45) (10906, 2)
(5372, 45) (5372, 2)


In [14]:
batch_size = 32
model.fit(X_train, Y_train, epochs = 10, batch_size=batch_size, verbose = 1)

Epoch 1/10
10906/10906 [==============================] - 39s 4ms/step - loss: 0.3892 - acc: 0.8082
Epoch 2/10
10906/10906 [==============================] - 36s 3ms/step - loss: 0.1746 - acc: 0.9299
Epoch 3/10
10906/10906 [==============================] - 41s 4ms/step - loss: 0.1351 - acc: 0.9444
Epoch 4/10
10906/10906 [==============================] - 41s 4ms/step - loss: 0.1116 - acc: 0.9531
Epoch 5/10
10906/10906 [==============================] - 39s 4ms/step - loss: 0.0920 - acc: 0.9613
Epoch 6/10
10906/10906 [==============================] - 42s 4ms/step - loss: 0.0774 - acc: 0.9677
Epoch 7/10
10906/10906 [==============================] - 41s 4ms/step - loss: 0.0648 - acc: 0.9714
Epoch 8/10
10906/10906 [==============================] - 38s 3ms/step - loss: 0.0601 - acc: 0.9733
Epoch 9/10
10906/10906 [==============================] - 38s 3ms/step - loss: 0.0523 - acc: 0.9782
Epoch 10/10
10906/10906 [==============================] - 39s 4ms/step - loss: 0.0464 - acc: 0.9791

In [16]:
model.evaluate(X_test,Y_test,batch_size=batch_size)

5372/5372 [==============================] - 3s 574us/step


[0.3172314937960853, 0.9287043930165197]